In [ ]:
for key in np.unique(np.where(loads == 0)[0]):
    print element_keys[key]
    for i in xrange(len(loads[key])):
        if loads[key][i] == 0: print i/12, i%12 + 8
    print '\n\n'

In [ ]:
df = pd.read_csv(os.path.join(raw_transaction_path, '03012016_03312016.csv'))
df = df.loc[df['ElementKey'] == 7266]
df['TransactionDateTime'] = pd.to_datetime(df['TransactionDateTime'])
df['Time'] = df['TransactionDateTime'].dt.time
df['Day'] = df['TransactionDateTime'].dt.weekday
df_ = df.loc[(df['Time'] <= datetime.time(18,0,0)) & (df['Time'] >= datetime.time(16,0,0))]
df_1 = df_.loc[df['Day'] == 1]
df_1

In [ ]:
figure_functions.create_animation(loads, gps_loc, N, P, fig_path, animation_path)

In [ ]:
with open(os.path.join(data_path, 'blockface_locs.p'), 'rb') as f:
    locations = pickle.load(f)

gmap = gmplot.GoogleMapPlotter(47.612676, -122.345028, 15)

for i in np.unique(np.where(loads == 0)[0]):
    gmap.marker(gps_loc[i, 0], gps_loc[i, 1], title=str(element_keys[i]))
gmap.draw(os.path.join(fig_path, 'test.html'))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
adjacency = np.zeros((len(block_keys), len(block_keys)))
N = len(block_keys)

for i in xrange(N):
    if block_keys[i] in north:
        zone1 = 'north'
    elif block_keys[i] in south:
        zone1 = 'south'
    else:
        print block_keys[i]
    for j in xrange(N):
        if i == j:
            continue
        
        if block_keys[j] in north:
            zone2 = 'north'
        elif block_keys[j] in south:
            zone2 = 'south'
        else:
            print block_keys[j]
        
        if zone1 == zone2:
            adjacency[i, j] = 1
        else:
            pass

In [ ]:
key = 1017
df = pd.read_csv(os.path.join(raw_transaction_path, '03012016_03312016.csv'))
df = df.loc[df['ElementKey'] == key]
df = df.sort_values(by='TransactionDateTime')
df['TransactionDateTime'] = pd.to_datetime(df['TransactionDateTime'])
df['Time'] = df['TransactionDateTime'].dt.time
df['Day'] = df['TransactionDateTime'].dt.weekday
df['DurationSeconds'] = df['PaidDuration'].values.astype(int)%60
df['DurationMinutes'] = df['PaidDuration'].values.astype(int)/60
df['DurationHours'] = df['DurationMinutes'].values.astype(int)/60
df['DurationMinutes'] = df['DurationMinutes'] - 60*df['DurationHours']
df['TransactionDateTimeEnd'] = df.apply(lambda row: row['TransactionDateTime'] + datetime.timedelta(hours=row['DurationHours'], minutes=row['DurationMinutes'], seconds=row['DurationSeconds']), axis=1)
df['TimeEnd'] = df['TransactionDateTimeEnd'].dt.time
df['Date'] = df['TransactionDateTime'].dt.date

df_ = df.loc[(df['Time'] <= datetime.time(11,0,0)) & (df['Time'] >= datetime.time(9,0,0))]
df_1 = df_.loc[df['Date'] == datetime.date(2016,3,3)]
for i in xrange(1, 59):
    print i, len(df_1.loc[(df_1['Time'] <= datetime.time(9,i,0)) & (df_1['TimeEnd'] >= datetime.time(9,i+1,0))])/9.

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
time = 58
run_all.run_figures(element_keys, loads, gps_loc, N, P, data_path, fig_path, animation_path, time)